# Previsões Crescimento Vegetativo Vibra

# Setup Inicial


## Bibliotecas utilizadas

In [35]:
import pandas as pd
import numpy as np
import json

from datetime import datetime
from dateutil.relativedelta import relativedelta

import logging
import warnings

from prophet import Prophet
from prophet.diagnostics import performance_metrics


# Medir performance dos modelos
from sklearn import metrics

# Visualização
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs import *

## Definições Gerais

In [36]:
warnings.filterwarnings('ignore')

In [37]:
# usado para remover as mensagens de output de processamento dos modelos prophet.
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [38]:
# definção dos diretórios relativos

# replace with your container name
blob_container_name = 'general' 

# replace with your relative folder path
blob_relative_path_raw = 'raw/mercado_potencial/'
blob_relative_path_enriched = 'enriched/mercado_potencial/'


# replace with your linked service name
linked_service_raw = 'LS_ADLS_RAW_01'
linked_service_enriched = 'LS_ADLS_ENRICHED_01'

ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]


dir_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
dir_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)


## Funções Auxiliares

In [39]:
def modelo_final (dados, periodos = 365, frequencia = 'd'):
    """Ajusta modelo prophet de serie temporal.

        Define, ajusta e prevê modelo de serie temporal baseado na biblioteca prophet.
        Modelo com a possibilidade de inclusão de variáveis regressoras. 
        
    Parâmetros
    ----------
    dados: pd.DataFrame 
        Dataframe com variáveis exigidas para a criação do modelo prophet.
        No dataframe deve conter, obrigatoriamente, coluna 'ds': datatime da série e 
        'y': variável alvo do modelo.
        As variáveis regressoras, quando existire, devem estar presentes no mesmo dataframe.
    periodos: int
        Períodos, em meses, a serem previstos pelo modelo.
        Por padrão, foi definido 12 meses a serem previstos.
    freq: string
        Frequência de criação dos periodos futuros a ser usado no predict.
        ['d': diária; 'b': dias utieis, 'M': mensal (último dia); 'MS': mensal (1º dia)]
        Referência: http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases

    
    Retorno
    -------
    previsao: pd.DataFrame
        Dataframe com os resultados do predict model adcionada das variáveis de input do modelo.
    modelo: prophet model
        Modelo prophet ajustado (fitted) 
    """

    #regressores = dados.columns.to_list()[2:]       # os 2 primeiros são DataTime e o Y da serie temporal

    lst_results = ['ds','yhat','yhat_lower', 'yhat_upper',
                   'trend', 'trend_lower', 'trend_upper',
                   'yearly', 'yearly_lower', 'yearly_upper']
    
    modelo = Prophet(seasonality_mode='multiplicative',
                     yearly_seasonality=True,
                     weekly_seasonality=True,
                     daily_seasonality=False,
                     changepoint_range=0.95, 
                     changepoint_prior_scale=0.1)

    # for i in range(len(regressores)):               # passeia por cada lista agregativa
    #     modelo.add_regressor(regressores[i], standardize=False)
                
    modelo.fit(dados)

    futuro = modelo.make_future_dataframe(periods=periodos, freq=frequencia) #MS início do mês - M fim do mês
    futuro = pd.merge(futuro, dados, on='ds', how='outer')
    futuro = futuro.fillna(method='ffill')
    
    previsao = modelo.predict(futuro)
    previsao = pd.merge(previsao[lst_results],dados, on='ds',how='outer')

    return(previsao, modelo)

In [40]:
def dividir_train_test(data, num_meses = 12):
    '''
    Função que divide os dados a serem usados pelos modelos em treino e teste conforme o
    padrão pd.DataFrame Prothet contendo as colunas ['ds','y']

    Parâmetros
    ----------
        data: pd.Dataframe
            dataframe em formato time series do Prothet a ser dividido
        num_meses: int
            número de meses usados para teste (default 12 meses)
    Retorno
    -------:
        train, test: pd.Dataframe
            Dataframes de treino e teste
    '''
    train_data_fim = data['ds'].max() - relativedelta(months=num_meses)
    # divisão dos datasets de Treino: train e Teste:test
    train = data[data['ds'] <= train_data_fim].reset_index(drop=True)
    test = data[data['ds'] > train_data_fim].reset_index(drop=True)

    return (train, test)

In [41]:
def grafico_volumetria (previsao , mape, titulo = 'Título do Gráfico', subtitulo = 'subtítulo do gráfico'):
    """Gera gráfico interativo em plotly.

        Gera gráfico do valor real e do valor predito, interativo criado com auxilio da biblioteca plotly. 
        
    Parâmetros
    ----------
    previsao: pd.DataFrame 
        Dataframe com variáveis `ds`: datatime, `y`: float ou int valores reais da série e 
        `yhat`: float ou int valores preditos pelo modelo
    titulo: string
        Título a ser plotado no gráfico.
    y_label: string
        Título do eixo y do gráfico 

    Retorno
    -------
    fig: plotly object
        Gáfico a ser apresentado ou salvo 
    """

    p1 = previsao.resample('MS', on='ds').sum().reset_index(drop=False) # mês
    p2 = previsao.resample('W', on='ds').sum().reset_index(drop=False) # semana
    
    p1.loc[p1['y']==0,'y'] = np.nan
    p2.loc[p2['y']==0,'y'] = np.nan


    
    fig = make_subplots(rows=3, cols=1,
                        y_title='Volume (m³)',
                        subplot_titles=('Mensal', 'Semanal','Diário'),
                        shared_yaxes=False,
                        vertical_spacing=0.05 )

        # grafico 01
    fig.add_trace(go.Scatter(
                x = p1['ds'],
                y = p1['yhat'],
                line = {'color': 'rgb(240,128,128)'},
                name = 'Predito',
                showlegend=True),
        row=1, col=1)

    fig.add_trace(go.Scatter(
                x = p1['ds'],
                y = p1['y'],
                line = {'color': 'rgb(100,149,237)'},
                name = 'Real',
                showlegend=True),
        row=1, col=1)

        # grafico 02
    fig.add_trace(go.Scatter(
                x = p2['ds'],
                y = p2['yhat'],
                line = {'color': 'rgb(240,128,128)'},
                name = 'Predito',
                showlegend=False),
        row=2, col=1)

    fig.add_trace(go.Scatter(
                x = p2['ds'],
                y = p2['y'],
                line = {'color': 'rgb(100,149,237)'},
                name = 'Real',
                showlegend=False),
        row=2, col=1)

        # grafico 03
    fig.add_trace(go.Scatter(
                x = previsao['ds'],
                y = previsao['yhat'],
                line = {'color': 'rgb(240,128,128)'},
                name = 'Predito',
                showlegend=False),
        row=3, col=1)

    fig.add_trace(go.Scatter(
                x = previsao['ds'],
                y = previsao['y'],
                line = {'color': 'rgb(100,149,237)'},
                name = 'Real',
                showlegend=False),
        row=3, col=1)


    fig.add_annotation(
                showarrow=False,
                text='Gerado em ' + datetime.now().strftime("%d %b %Y às %H:%M:%S"),
                font=dict(size=12),
                xref='paper',
                x=1.1,
                yref='paper',
                y=-0.12)
    
    fig.add_annotation(
                showarrow=False,
                text='MAPE (acumulado 12 meses): {:.2f}%'.format(mape),
                font=dict(size=14),
                xref='paper',
                x=0.0,
                yref='paper',
                y=-0.12)

    fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.03)
    fig.update_xaxes(rangeslider= {'visible':False}, row=1, col=1)
    fig.update_xaxes(rangeslider= {'visible':False}, row=2, col=1)


    fig.update_layout(
                title = titulo + '<br><sup>'+ subtitulo + '</sup>',
                #xaxis_title = "Período",
                #yaxis_title = "y_label",
                legend_title = "Legenda",
                font=dict(
                    family = "Courier New, monospace",
                    size = 14,
                    color = "royalblue"),
                height=1200
    )

    return(fig)


# Análise Exploratória


## Coletando dados históricos de vendas da VIBRA

In [42]:
cols = ['VEDI_VOL', 'VEDI_DAT_VEN', 'VEDI_COD_CLI', 'CLIE_NUM_CPF_CNPJ', 'CLIE_NOM_CLIENTE', 'AECO']
col_names = ['vol_venda','data_venda', 'cod_cliente', 'cpf_cnpj', 'nm_cliente', 'ativ_economica']

In [43]:
#arquivo = dir_raw + 'volumetria/hist_diesel.csv'
arquivo = dir_enriched + 'histdiesel/hist_diesel.csv'
hist_diesel = pd.read_csv(arquivo,
                          usecols = cols,
                          sep=',',
                          decimal='.',
                          encoding='utf-8',
                          storage_options = {'linked_service' : linked_service_enriched})

# hist_diesel = pd.read_parquet(dir_enriched + 'histdiesel/hist_diesel.parquet',
#                           storage_options = {'linked_service' : linked_service_enriched})


## Ajustes Histórico Vendas

In [44]:
hist_diesel.columns = col_names

# replace = { 'PASSAG'        :'PASSAGEIRO',
#             'MINERAÃÃO'   :'MINERACAO',
#             'INDÃSTRIAS'   :'INDUSTRIA',
#             'CONSTRUÃÃO'  : 'CONSTRUCAO',
#             'FERROVIÃR'    : 'FERROVIARIO',
#             'MARÃTIMO'     : 'MARITIMO',
#             'QUÃMICA'      : 'QUIMICA',
#             'TÃRMICA'      : 'TERMICA',
#             'ÃLEO'         : 'OLEO',
#             'GÃS'          : 'GAS',
#             'AVIAÃÃO'     : 'AVIACAO'
#             }

# hist_diesel.replace({'ativ_economica': replace}, regex=True, inplace=True)
hist_diesel['data_venda'] = pd.to_datetime(hist_diesel['data_venda'], format='%Y-%m-%d')
hist_diesel[['nm_cliente','ativ_economica']]   = hist_diesel[['nm_cliente','ativ_economica']].astype('string')

In [45]:
# agrupa volume por atividade economica
diesel = pd.DataFrame(hist_diesel.groupby(['ativ_economica', 'data_venda'])['vol_venda'].sum().reset_index())

## Verifica se a frequencia de vendas está em dias úteis


In [46]:
diesel['weekday'] = diesel['data_venda'].dt.day_name()
diesel['weekday'].value_counts()
# existem vendas em finais de semana...

## Verifica se existe venda negativa

In [47]:
diesel[diesel['vol_venda']<0]

In [48]:
# ajustando as vendas negativas
diesel.loc[diesel['vol_venda']<=0, 'vol_venda'] = 0

## Verificando a presença de vendas zero por dias da semana

testar o desempenho sem os valores zerados - remover os dias ou imputar nan

In [49]:
diesel[diesel['vol_venda']==0]['weekday'].value_counts()

# Proporções entre as Atividades Econômicas

## Agregar ConsFinal somando todos os volumes <br>revovendo TRR

In [50]:
# Incluindo Consumidor final = Todas as Atividades - TRR
filtro = diesel['ativ_economica']=='TRR'

cons = pd.DataFrame(diesel[~filtro].groupby(['data_venda']).sum().reset_index())
cons['ativ_economica'] = 'CONSUMIDOR FINAL'

order = [2,0,1]
cons = cons[cons.columns[order]]

diesel = pd.concat([diesel.iloc[:,0:3], cons], axis=0, ignore_index=True)

del(filtro, cons)

## Mudar formato da tabela para widetabe

In [51]:
#Widetable
diesel2 = diesel.pivot(index='data_venda', columns='ativ_economica')
diesel2.columns = diesel2.columns.get_level_values(1)
diesel2.columns.name = None

## Agregando vendas diárias por mês

In [52]:
#agregação mensal
diesel2m = diesel2.resample('MS').sum()#.reset_index(drop=False) # mês


## Salvando os volumes agrupados por mês

In [53]:
diesel2m.to_parquet(dir_enriched + 'volumetria/vol01_vendas_vibra_mes.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

## Proporção as Atividades Econômicas em Consumidor Final 

In [54]:
# prroporção consumidor final
filtro = ~diesel2m.columns.str.contains('TRR') # remover TRR da conta
filtro = diesel2m.columns[filtro]
cf_prop = diesel2m[filtro].div(diesel2m['CONSUMIDOR FINAL'].values,axis=0)

## Salvando as proporções do Consumidor Final

In [55]:
cf_prop.to_parquet(dir_enriched + 'volumetria/vol01_prop_consfinal_vegetativo.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

## Proporção de TRR e Consumidor Final

In [56]:
#proporção trr consumidor final
prop = diesel2m[['TRR','CONSUMIDOR FINAL']]
prop['total'] = prop['TRR'] + prop['CONSUMIDOR FINAL']
prop['TRRp'] = prop['TRR'].div(prop['total'].values,axis=0)
prop['CFp'] = prop['CONSUMIDOR FINAL'].div(prop['total'].values,axis=0)


In [57]:
prop.to_parquet(dir_enriched + 'volumetria/vol01_prop_trr-consfinal_vegetativo.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

# Modelos de Previsão de Vendas de Diesel VIBRA 

## Análise Exploratória dos Modelos


### Coletando lista de Atividades Econômicas

In [58]:
# lista atividades economicas
lista = np.unique(diesel['ativ_economica'].values)
lista

### Performance dos Modelos pra cada Atividade Econômica

In [59]:
cols = ['Atividade','MAPE']
mape = pd.DataFrame(columns=cols)
#lista = lista[3:4]
for i, j in enumerate(lista):
    filtro = diesel['ativ_economica']==lista[i]
    diesel_f = diesel[filtro].set_index('data_venda')
    diesel_f = diesel_f[diesel_f.columns[[1,0]]]
    diesel_f.reset_index(drop=False, inplace=True)
    diesel_f.rename({'data_venda':'ds','vol_venda':'y'}, axis=1, inplace=True)

    treino, teste = dividir_train_test(diesel_f, num_meses=12)

    previsao, modelo = modelo_final(dados=treino, periodos=365, frequencia='d') # modelo ajustado para prever informação diária

    previsao.loc[previsao['yhat']<=0, 'yhat'] = 0 # ajuste de previsão - sem previsão negativa

    a = teste.iloc[:,0:2]
    b = previsao[previsao['y'].isnull()].iloc[:,0:2].reset_index(drop=True)
    b.columns = ['ds','yhat']
    c = pd.merge(a,b, how='inner') # diário
    c['dif'] = c['yhat'] - c['y']

    mape_ano = abs(c['dif'].sum()/c['y'].sum())*100

    mape.loc[-1] = [lista[i],mape_ano]
    mape.reset_index(drop=True, inplace=True)

    a1 = pd.concat([treino.iloc[:,0:2], teste.iloc[:,0:2]],ignore_index=True) 
    b1 = previsao.reset_index(drop=True)
    p1 = pd.merge(a1,b1, how='outer')

    p2 = previsao.resample('MS', on='ds').sum().reset_index(drop=False) # mês
    p3 = previsao.resample('W', on='ds').sum().reset_index(drop=False) # semana

    p2.loc[p2['y']==0,'y'] = np.nan
    p3.loc[p3['y']==0,'y'] = np.nan
 
    ## PARA VISUALIZAR OS GRÁFICOS REMOVER COMENTATRIOS ABAIXO
    # graf = grafico_volumetria(  p1, 
    #                            mape_ano,
    #                            'Previsão Volumetria - ' + lista[i],
    #                            'treino: 01.01.2016 - 30.11.2021')
    
    # graf = plotly.offline.plot(graf, output_type='div')
    # displayHTML(graf)


### Resumo MAPE das atividades

In [60]:
mape_ = mape.copy()
mape_['MAPE'] = round(mape_['MAPE'],2)
mape_ = mape_.sort_values('MAPE').reset_index(drop=True)
mape_

### Salvando dataframe MAPE em formato html
* não foi possivel no synapse

In [61]:
# mape_ = mape.copy()
# mape_['MAPE'] = round(mape_['MAPE'],2)
# mape_ = mape_.sort_values('MAPE').reset_index(drop=True)
# mape_['Atividade'] = mape_['Atividade'].apply(lambda x: "<a href='volumetria_{}.html'>{}</a>".format(x,x))

# mape_.columns = ['Atividade Econômica', 'MAPE (%)']


# pd.set_option('colheader_justify', 'center')   

# html_string = '''
# <html>
#   <head><title>Volumetria VIBRA</title></head>
#   <link rel="stylesheet" type="text/css" href="df_style.css"/>
#   <body>
#     <div id="cabecalho">

#     <div>
#     <div id="conteudo">
#       <h1> Performance dos modelos de previsão para volumetria</h1>
#       <p> Erro Percentual Absoluto (MAPE) dos modelos de previsão específicos para cada atividade econômica da Vibra Energia.</p>
#       <p> Mape calculado com base no acumulado dos últimos 12 meses da série.</p>
#       <br> </br>
#       {table}
#     <div>
#     <div id="rodape">
      
#     <div>
#   </body>
# </html>.
# '''

# # saida do dataframe em formato html 
# with open('outputs/volumetria/00_mape.html', 'w') as f:
#     f.write(html_string.format(table=mape_.to_html(classes='mystyle',
#                                                   render_links=True,
#                                                   escape=False)))


## Previsao dos modelos por atividade econômica

Salvando em enriched as previões de vendas vibra 12 meses a frente

In [62]:
prev = pd.DataFrame()

for i, j in enumerate(lista):
    
    filtro = diesel['ativ_economica']==lista[i]
    diesel_f = diesel[filtro].set_index('data_venda')
    diesel_f = diesel_f[diesel_f.columns[[1,0]]]
    diesel_f.reset_index(drop=False, inplace=True)
    diesel_f.rename({'data_venda':'ds','vol_venda':'y'}, axis=1, inplace=True)
   

    previsao, modelo = modelo_final(dados=diesel_f, periodos=365, frequencia='d') # modelo ajustado para prever informação diária

    previsao.loc[previsao['yhat']<=0, 'yhat'] = 0 # ajuste de previsão - sem previsão negativa

    b1 = previsao.reset_index(drop=True)
    p1 = pd.merge(diesel_f,b1, how='outer')

    p2 = p1.resample('MS', on='ds').sum().reset_index(drop=False) # mês
    p2.loc[p2['y']==0,'y'] = np.nan
    p2['Atv_Economica'] = lista[i]

    prev = pd.concat([prev,p2], ignore_index=True)

prev.to_parquet(dir_enriched + 'volumetria/vol01_prev_vegetativo.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

# Acesso à Documentação

In [63]:
help(modelo_final)

In [64]:
help(grafico_volumetria)